In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [2]:
# create spark session

spark = SparkSession.builder.appName("test").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/23 18:37:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [105]:
# read data from csv file

schema = StructType(
    [
        StructField("popup_name", StringType(), True),
        StructField("blog_post_url", StringType(), True),
        StructField("popup_version|start_date|popup_category", StringType(), True),
        StructField("popup_header", StringType(), True),
        StructField("popup_description", StringType(), True),
        StructField("popup_image_url", StringType(), True),
        StructField("popup_title", StringType(), True),
        StructField("views", IntegerType(), True),
        StructField("registrations", IntegerType(), True)
    ]
)

df = spark.read.option("delimiter", "\t").option("multiline", "true") \
    .option("quote", '"') \
    .option("header", "true") \
    .option("escape", "\\") \
    .option("escape", '"') \
    .csv("data/dataset.tsv", header=True, schema=schema) \
    .withColumnRenamed("popup_version|start_date|popup_category", "array_fields")

In [106]:
df.count()

2063

In [107]:
# cast array_fields to array type and explode it to get popup_version, start_date, popup_category

df = df.withColumn("array_fields", split(regexp_replace("array_fields", "[\[\]]", ""), "\",\"")) \
    .withColumn("popup_version", regexp_replace(col("array_fields")[0], '"', "")) \
    .withColumn("start_date", regexp_replace(col("array_fields")[1], '"', "")) \
    .withColumn("popup_category", regexp_replace(col("array_fields")[2], '"', "")) \
    .drop("array_fields")

In [167]:
df0 = df.select(sum("views").alias("total_views"), sum("registrations").alias("total_registrations"))

In [168]:
df0.show()

+-----------+-------------------+
|total_views|total_registrations|
+-----------+-------------------+
|    2651866|              47432|
+-----------+-------------------+



In [177]:
47432/2651866

0.01788627328831849

In [185]:
# group popup_category and get sum of views and registrations

df1 = df.groupBy("popup_category") \
    .agg(sum("views").alias("views"),
         sum("registrations").alias("registrations"),
         round(sum("registrations") / sum("views"), 3).alias("conversion_rate"),
         round(sum("registrations") / 47432, 3).alias("registration_rate")) \
    .orderBy(desc("registration_rate"))

# group popup_version and get sum of views and registrations

df2 = df.groupBy("popup_version") \
    .agg(sum("views").alias("views"),
         sum("registrations").alias("registrations"),
         round(sum("registrations") / sum("views"), 3).alias("conversion_rate"),
         round(sum("registrations") / 47432, 3).alias("registration_rate")) \
    .orderBy(desc("registration_rate"))

# group start_date and get sum of views and registrations

df3 = df.groupBy("start_date") \
    .agg(sum("views").alias("views"),
         sum("registrations").alias("registrations"),
         round(sum("registrations") / sum("views"), 3).alias("conversion_rate"),
         round(sum("registrations") / 47432, 3).alias("registration_rate")) \
    .orderBy(desc("registration_rate"))

# group popup_name and get sum of views and registrations

df4 = df.groupBy("popup_name") \
    .agg(sum("views").alias("views"),
         sum("registrations").alias("registrations"),
         round(sum("registrations") / sum("views"), 3).alias("conversion_rate"),
         round(sum("registrations") / 47432, 3).alias("registration_rate")) \
    .orderBy(desc("registration_rate"))

# group blog_post_url and get sum of views and registrations

df5 = df.groupBy("blog_post_url") \
    .agg(sum("views").alias("views"),
         sum("registrations").alias("registrations"),
         round(sum("registrations") / sum("views"), 3).alias("conversion_rate"),
         round(sum("registrations") / 47432, 3).alias("registration_rate")) \
    .orderBy(desc("registration_rate"))

In [186]:
df1.show(10, False)

+-----------------------------------------------+------+-------------+---------------+-----------------+
|popup_category                                 |views |registrations|conversion_rate|registration_rate|
+-----------------------------------------------+------+-------------+---------------+-----------------+
|Perfecting your Craft                          |845810|20685        |0.024          |0.436            |
|generator                                      |887481|10822        |0.012          |0.228            |
|Understanding Publishing                       |280497|8127         |0.029          |0.171            |
|Perfecting your Craft, Understanding Publishing|52276 |1617         |0.031          |0.034            |
|Book Design                                    |75068 |1512         |0.02           |0.032            |
|Book Marketing, Understanding Publishing       |21833 |1120         |0.051          |0.024            |
|discovery                                      |381038

In [187]:
df2.show(100,False)

+-------------+-------+-------------+---------------+-----------------+
|popup_version|views  |registrations|conversion_rate|registration_rate|
+-------------+-------+-------------+---------------+-----------------+
|             |333606 |16709        |0.05           |0.352            |
|A            |1117711|15834        |0.014          |0.334            |
|B            |1200549|14889        |0.012          |0.314            |
+-------------+-------+-------------+---------------+-----------------+



In [193]:
df3.show(10,False)

+----------+------+-------------+---------------+-----------------+
|start_date|views |registrations|conversion_rate|registration_rate|
+----------+------+-------------+---------------+-----------------+
|2020-06-05|690452|8246         |0.012          |0.174            |
|2020-05-05|268394|6055         |0.023          |0.128            |
|2019-12-13|54754 |5670         |0.104          |0.12             |
|2020-05-06|91175 |2933         |0.032          |0.062            |
|2019-10-22|64729 |2338         |0.036          |0.049            |
|2019-10-16|22757 |2240         |0.098          |0.047            |
|2020-05-01|163226|2016         |0.012          |0.043            |
|2020-05-07|196504|2044         |0.01           |0.043            |
|2020-05-26|213101|1687         |0.008          |0.036            |
|2019-10-10|7511  |1141         |0.152          |0.024            |
+----------+------+-------------+---------------+-----------------+
only showing top 10 rows



In [192]:
df4.show(10,False)

+-------------------------------------+------+-------------+---------------+-----------------+
|popup_name                           |views |registrations|conversion_rate|registration_rate|
+-------------------------------------+------+-------------+---------------+-----------------+
|Character Profile Checklist 4        |90895 |7007         |0.077          |0.148            |
|Learning | Writing a Novel | 2020-03 |388304|4767         |0.012          |0.101            |
|Upgrade | Character Profile | 2020-03|281715|2989         |0.011          |0.063            |
|Worldbuilding Template               |26901 |2058         |0.077          |0.043            |
|Manuscript Template 2 (New Style)    |30114 |1673         |0.056          |0.035            |
|Story Structure Template 1           |50582 |1519         |0.03           |0.032            |
|Learning | Short Story | 2020-03     |162764|1358         |0.008          |0.029            |
|Character Profile Checklist 3        |46228 |1351

In [191]:
df5.show(10,False)

+-----------------------------------------------------+-----+-------------+---------------+-----------------+
|blog_post_url                                        |views|registrations|conversion_rate|registration_rate|
+-----------------------------------------------------+-----+-------------+---------------+-----------------+
|https://blog.bookly.com/character-profile/           |34069|5439         |0.16           |0.115            |
|https://blog.bookly.com/worldbuilding-guide/         |17346|1918         |0.111          |0.04             |
|https://blog.bookly.com/writing-contests/            |30128|1603         |0.053          |0.034            |
|https://blog.bookly.com/how-to-write-a-novel/        |30842|1337         |0.043          |0.028            |
|https://blog.bookly.com/how-to-write-a-book-proposal/|8267 |1176         |0.142          |0.025            |
|https://blog.bookly.com/creative-writing-prompts/    |64225|1099         |0.017          |0.023            |
|https://b

In [194]:
df1.select('popup_category').distinct().count()

23

In [198]:
# group by all columns and get sum of views and registrations and filter by conversion_rate 1.0

df6 = df.groupBy("popup_name", "blog_post_url", "popup_version", "start_date", "popup_category", "popup_header",
                 "popup_description", "popup_image_url", "popup_title") \
    .agg(sum("views").alias("views"),
         sum("registrations").alias("registrations"),
         round(sum("registrations") / sum("views"), 3).alias("conversion_rate"),
         round(sum("registrations") / 47432, 3).alias("registration_rate")) \
    .orderBy(desc("conversion_rate")) \
    .filter(col("conversion_rate") == 1.0)

In [201]:
df6.show(18,False)

+--------------------------------------+------------------------------------------------------------------------------------+-------------+----------+------------------------+------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------+-----+-------------+---------------+-----------------+
|popup_name                            |blog_post_url                                                                       |popup_version|start_date|popup_category          |popup_header                                          |popup_description                                                                                                                     